<a href="https://colab.research.google.com/github/jieun0441/DeeplearningStudy/blob/main/Converter_for_Google_Colab_txt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Your Google Drive First

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install `json-flat` library

In [ ]:
!pip install json-flat

ERROR: Operation cancelled by user


# Import the libraries and run all the cells

In [ ]:
import csv
import json
import os
import sys

from jsonflat import JsonFlat

ModuleNotFoundError: ignored

In [ ]:
def getFileList(root):
    fileList = []
    for path, dirs, files in os.walk(root):
        for file in files:
            if file.lower().endswith('.json'):
                fileList.append(os.path.join(path, file))
    return fileList

In [ ]:
def mkdir_recursive(path):
    sub_path = os.path.dirname(path)
    if not os.path.exists(sub_path):
        mkdir_recursive(sub_path)
    if not os.path.exists(path):
        os.mkdir(path)

In [ ]:
def readJSONFile(input_file):
    data = []
    with open(input_file, 'r', encoding="utf-8") as f:
        for row in f:
            if row.strip().startswith("[") or row.strip().endswith(']'):
                # Skip this row
                continue
            else:
                try:
                    temp_row = row.strip()
                    temp_row = temp_row.rstrip(",")
                    temp_row = temp_row.rstrip("[")
                    temp_row = temp_row.rstrip("]")
                    temp_row = temp_row.rstrip(",")
                    data.append(json.loads(temp_row))
                except Exception as e:
                    print("[-] {}".format(e))
                    print("[-] Skipping this file due to above error")
                    return None
    return data

In [ ]:
def run(split_files, row_limit, input_folder_path, output_folder_path):
    print("[+] Reading input files from given path")
    files = getFileList(input_folder_path)
    print("[+] Total file paths fetched: ", len(files))
    print()
    for num, file in enumerate(files):
        try:
            print("[+] Reading file {}: {}".format(num + 1, file))
            data = readJSONFile(file)
            if data is None:
                print("[-] Moving on to next file")
                print()
                continue
            print("[+] Parsing into csv")
            flatten_data = JsonFlat().flatten(data)
            fieldnames = flatten_data['field_names']
            flattened_rows = flatten_data['rows']
            print("[+] Preparing paths for output directories")
            relPath = os.path.relpath(file, input_folder_path)
            temp_abs_path = os.path.join(output_folder_path, relPath)
            temp_folder_path = os.path.dirname(temp_abs_path)
            if not os.path.exists(temp_folder_path):
                print("[+] Creating output directories")
                mkdir_recursive(temp_folder_path)
            if split_files:
                print("[+] Writing data to multiple smaller output files")
                current_piece = 1
                current_limit = row_limit
                filename = os.path.basename(temp_abs_path).split(".")[0] + "-Part-{}.csv".format(current_piece)
                output_file_path = os.path.join(temp_folder_path, filename)
                f_out = open(output_file_path, 'w', newline='', encoding='utf-8')
                current_out_writer = csv.DictWriter(f_out, fieldnames=fieldnames)
                current_out_writer.writeheader()
                print(filename)
                for row_index, row in enumerate(flattened_rows):
                    if row_index + 1 > current_limit:
                        f_out.flush()
                        f_out.close()
                        current_piece += 1
                        current_limit = row_limit * current_piece
                        filename = os.path.basename(temp_abs_path).split(".")[0] + "-Part-{}.csv".format(current_piece)
                        output_file_path = os.path.join(temp_folder_path, filename)
                        f_out = open(output_file_path, 'w', newline='', encoding='utf-8')
                        current_out_writer = csv.DictWriter(f_out, fieldnames=fieldnames)
                        current_out_writer.writeheader()
                        print(filename)

                    current_out_writer.writerow(row)
            else:
                print("[+] Writing data to a single output file")
                filename = os.path.basename(temp_abs_path).split(".")[0] + ".csv"
                output_file_path = os.path.join(temp_folder_path, filename)
                with open(output_file_path, "w", encoding="utf-8", newline="") as f_out:
                    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
                    writer.writeheader()
                    writer.writerows(flattened_rows)
                print("[+] Output file successfully written to: ", output_file_path)
            print()
            del flatten_data
            del data
        except Exception as e:
            print("[-] {}".format(e))
            print("[-] Skipping this file due to above error")
            print()

# Main

Change the paths in this cell and run it only after executing all the above cells.

In [ ]:
try:
    input_folder_path = r"/content/drive/MyDrive/PropertyGuru"
    output_folder_path = r"/content/drive/MyDrive/PropertyGuru/Output"
    split_files = input("Do you want to split the output files into smaller files? (y/n): ")
    split_files = True if split_files.strip().lower() == "y" else False
    row_limit = None
    if split_files:
        row_limit = input(
            "Enter the number of records you want the file to split in\nfor e.g. 100000, 2000000, 350000...: ")
        if row_limit.strip().isnumeric():
            row_limit = int(row_limit)
        else:
            print("Please enter an integer value for row split count!")
            sys.exit(0)
    run(split_files, row_limit, input_folder_path, output_folder_path)
    print("\n[+] All files converted successfully")
except Exception as e:
    print(e)
    print("[-] Stopping executiong due to above error!")

Do you want to split the output files into smaller files? (y/n): n
[+] Reading input files from given path
[+] Total file paths fetched:  44

[+] Reading file 1: /content/drive/MyDrive/PropertyGuru/2020-09-29/sale/property_urls.json
[+] Parsing into csv
[+] Preparing paths for output directories
[+] Creating output directories
[+] Writing data to a single output file
[+] Output file successfully written to:  /content/drive/MyDrive/PropertyGuru/Output/2020-09-29/sale/property_urls.csv

[+] Reading file 2: /content/drive/MyDrive/PropertyGuru/2020-09-29/sale/property_data.json
[+] Parsing into csv
[+] Preparing paths for output directories
[+] Writing data to a single output file
[+] Output file successfully written to:  /content/drive/MyDrive/PropertyGuru/Output/2020-09-29/sale/property_data.csv

[+] Reading file 3: /content/drive/MyDrive/PropertyGuru/2020-09-29/rent/property_urls.json
[+] Parsing into csv
[+] Preparing paths for output directories
[+] Creating output directories
[+] Wri